In [1]:
import numpy as np

In [2]:
import os
import ctypes
import sys


modules_to_clean = []
for name in list(sys.modules.keys()):
    if any(substring in name for substring in ['h5py', 'prometheus', '_hdf5', '_h5py', '_errors']):
        modules_to_clean.append(name)

for mod in modules_to_clean:
    if mod in sys.modules:
        del sys.modules[mod]

# library path
intel_paths = [
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64",
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/mkl/lib/intel64",
]

current_ld_path = os.environ.get('LD_LIBRARY_PATH', '')
new_ld_path = ':'.join(intel_paths + [current_ld_path])
os.environ['LD_LIBRARY_PATH'] = new_ld_path

intel_libs = [
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libimf.so",
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libintlc.so.5",
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libsvml.so",
]

for lib_path in intel_libs:
    try:
        ctypes.CDLL(lib_path, mode=ctypes.RTLD_GLOBAL)
        print(f"Successfully preloaded: {lib_path}")
    except Exception as e:
        print(f"Failed to preload {lib_path}: {e}")

# # Python paths
sys.path.insert(0, '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus')
sys.path.insert(0, '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples')

try:
    import h5py
    print("✓ h5py imported successfully")
except Exception as e:
    print(f"✗ h5py import failed: {e}")

try:
    from prometheus import Prometheus, config
    print("✓ prometheus imported successfully!")
except Exception as e:
    print(f"✗ prometheus import failed: {e}")
    import traceback
    traceback.print_exc()

Successfully preloaded: /software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libimf.so
Successfully preloaded: /software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libintlc.so.5
Successfully preloaded: /software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libsvml.so
✓ h5py imported successfully
✓ prometheus imported successfully!


In [3]:
import pandas as pd
import numpy as np
import time
import sys
import argparse
import os
import logging
from datetime import datetime

sys.path.append('../')
from prometheus import Prometheus, config
import prometheus
import gc

# Import our utility functions
from genie_parser_injection import parse_and_convert_genie

from inject_in_cylinder import inject_particles_in_cylinder
from rotate_particles import rotate_particles_final
from aggregate_hadronic_shower import group_hadronic_showers

# Set up logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
RESOURCE_DIR = f"{'/'.join(prometheus.__path__[0].split('/')[:-1])}/resources/"
OUTPUT_DIR = f"{'/'.join(prometheus.__path__[0].split('/')[:-1])}/genie_examples/output"


In [4]:
simset = 49
root_file_path = '/groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_10000.gtac.root'


start_time = time.time()


print(f"Using simset: {simset}")
print(f"Using root file: {root_file_path}")

# Timing for file processing and conversion in one step
processing_start_time = time.time()
prometheus_set, primary_set = parse_and_convert_genie(root_file_path)
processing_end_time = time.time()
print(f"File processing and conversion completed in {processing_end_time - processing_start_time:.2f} seconds")

# absolute path:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# output_dir = f"/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/examples/output"
primrary_file_path = f'{OUTPUT_DIR}/genie_events_primary_new_test.parquet'
prometheus_file_path = f'{OUTPUT_DIR}/genie_events_prometheus_new_test.parquet'

# inject into cylinder:
primary_set, prometheus_set = inject_particles_in_cylinder(primary_set, ## neutrino information
                    prometheus_set, ## child particle information
                    cylinder_radius=500, ## meters
                    cylinder_height = 1000, # meters
                    cylinder_center = (0, 0, 0), # meters not including offset
                    )

primary_set, prometheus_set = rotate_particles_final(primary_set, prometheus_set)


# Timing for serialization and saving
save_start_time = time.time()
# position arrays to serialized strings
#prometheus_set['position'] = prometheus_set['position'].apply(lambda x: [arr.tolist() for arr in x])
# num_events = len(prometheus_set)
# print(f"Processing {num_events} events")
# prometheus_set.to_parquet(prometheus_file_path)
# primary_set.to_parquet(primrary_file_path)
# save_end_time = time.time()
# print(f"Serialization and saving completed in {save_end_time - save_start_time:.2f} seconds")

# ## GENIE stuff:
# config["injection"]["name"] = "GENIE"
# config["run"]["outfile"] = f"{OUTPUT_DIR}/100_events_{timestamp}.parquet"
# config["run"]["nevents"] = num_events
# config["injection"]["GENIE"] = config["injection"].get("GENIE", {})
# config["injection"]["GENIE"]["paths"] = config["injection"]["GENIE"].get("paths", {})
# config["injection"]["GENIE"]["inject"] = False ## we aren't injecting using prometheus, we are using an injection file
# config["injection"]["GENIE"]["simulation"] = {}
# ## geofile:
# config["detector"]["geo file"] = f"{RESOURCE_DIR}/geofiles/icecube.geo"
# ## ppc configuration:
# config['photon propagator']['name'] = 'PPC_UPGRADE'
# config["photon propagator"]["PPC_UPGRADE"]["paths"]["ppc_tmpdir"] = "./ppc_tmpdir"+str(simset)
# config["photon propagator"]["PPC_UPGRADE"]["paths"]["ppc_tmpfile"] = "ppc_tmp"+str(simset)
# config["photon propagator"]["PPC_UPGRADE"]["simulation"]["supress_output"] = False ## for printing!

# # Timing for Prometheus simulation
# sim_start_time = time.time()
# p = Prometheus(config, primary_set_parquet_path=primrary_file_path, prometheus_set_parquet_path=prometheus_file_path)
# p.sim()
# sim_end_time = time.time()
# print(f"Prometheus simulation completed in {sim_end_time - sim_start_time:.2f} seconds")

# # End timing
# end_time = time.time()
# total_time = end_time - start_time
# print(f"Total execution time: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")
# print('Finished without catastrophic error')
# return



2025-05-15 12:02:42,691 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_10000.gtac.root
2025-05-15 12:02:42,692 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_10000.gtac.root
2025-05-15 12:02:42,695 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_10000.gtac.root
2025-05-15 12:02:42,695 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_10000.gtac.root
2025-05-15 12:02:42,708 - asyncio - DEBUG - Using selector: EpollSelector
2025-05-15 12:02:42,709 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_10000.gtac.root
2025-05-15 12:02:42,710 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_10000.gtac.root
2025-05-15 12:02:42,713 - fsspec.local

Using simset: 49
Using root file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_10000.gtac.root


2025-05-15 12:02:42,722 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_10000.gtac.root
2025-05-15 12:02:42,723 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_10000.gtac.root
2025-05-15 12:02:42,733 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_10000.gtac.root
2025-05-15 12:02:42,735 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_10000.gtac.root
2025-05-15 12:02:42,736 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_10000.gtac.root
2025-05-15 12:02:42,737 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_10000.gtac.root
2025-05-15 12:02:42,738 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_ghep

File processing and conversion completed in 8.31 seconds


In [5]:
prometheus_set  = group_hadronic_showers(prometheus_set)

In [6]:
prometheus_set

,primary,e,pdg_code,interaction,theta,phi,bjorken_x,bjorken_y,pos_x,pos_y,pos_z,position,column_depth,custom_info
0,"[False, False]","[45.50518012924548, 10.86193159202349]","[13, -2000001006]","[CC, CC]","[2.067075661595442, 2.132042992339948]","[2.5511801288013594, 2.552868822758729]","[-1.0, -1.0]","[-1.0, -1.0]","[-282.10062345064307, -282.10062345064307]","[-235.13657087895137, -235.13657087895137]","[131.77952974883124, 131.77952974883124]","[[-282.10062345064307, -235.13657087895137, 13...","[-1.0, -1.0]","[child, child]"
1,"[False, False]","[1.553154902475276, 18.83828493494808]","[13, -2000001006]","[CC, CC]","[1.257652221818758, 0.8621987961995335]","[-0.07175560329268144, -0.4136971854003417]","[-1.0, -1.0]","[-1.0, -1.0]","[-266.6133132787735, -266.6133132787735]","[392.7532243368922, 392.7532243368922]","[428.3103116051809, 428.3103116051809]","[[-266.6133132787735, 392.7532243368922, 428.3...","[-1.0, -1.0]","[child, child]"
2,"[False, False]","[22.8860991194759, 6.540648101012984]","[14, -2000001006]","[NC, NC]","[1.0591866668147263, 0.9959436495723902]","[1.829713461036645, 1.4610488297749082]","[-1.0, -1.0]","[-1.0, -1.0]","[-190.06969725113595, -190.06969725113595]","[197.51769225223495, 197.51769225223495]","[397.9639690379437, 397.9639690379437]","[[-190.06969725113595, 197.51769225223495, 397...","[-1.0, -1.0]","[child, child]"
3,"[False, False]","[0.6646495645253374, 31.71207236002862]","[14, -2000001006]","[NC, NC]","[1.9034802913019888, 2.5164707547434686]","[0.18215455990949517, -0.2523157785774759]","[-1.0, -1.0]","[-1.0, -1.0]","[-410.56053905574043, -410.56053905574043]","[172.53365903631823, 172.53365903631823]","[220.2599909337447, 220.2599909337447]","[[-410.56053905574043, 172.53365903631823, 220...","[-1.0, -1.0]","[child, child]"
4,"[False, False]","[22.03573692056882, 4.220914829181715]","[14, -2000001006]","[NC, NC]","[1.663206122804835, 1.320453441894709]","[-2.9850900846065826, 3.0611679749533054]","[-1.0, -1.0]","[-1.0, -1.0]","[-101.89797162743382, -101.89797162743382]","[147.4691008193184, 147.4691008193184]","[-199.4574005302444, -199.4574005302444]","[[-101.89797162743382, 147.4691008193184, -199...","[-1.0, -1.0]","[child, child]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,"[False, False]","[27.600063236991506, 3.637189376367924]","[13, -2000001006]","[CC, CC]","[0.5756598304985038, 0.7170793984178334]","[-0.12885668520129273, -0.10395841486766698]","[-1.0, -1.0]","[-1.0, -1.0]","[-311.9725145069959, -311.9725145069959]","[95.77376614555551, 95.77376614555551]","[490.96665858950723, 490.96665858950723]","[[-311.9725145069959, 95.77376614555551, 490.9...","[-1.0, -1.0]","[child, child]"
9996,"[False, False]","[28.423240903483006, 11.059652493790791]","[13, -2000001006]","[CC, CC]","[0.9820697935318712, 0.9132725985913972]","[-2.828069968541139, -2.8870558592753808]","[-1.0, -1.0]","[-1.0, -1.0]","[349.29692694155557, 349.29692694155557]","[-61.512554712957815, -61.512554712957815]","[-285.00734403323037, -285.00734403323037]","[[349.29692694155557, -61.512554712957815, -28...","[-1.0, -1.0]","[child, child]"
9997,"[False, False]","[34.563909769942185, 13.695911597641514]","[13, -2000001006]","[CC, CC]","[1.0678033860163625, 1.0858090499877906]","[-2.3897710301020365, -2.383373459788714]","[-1.0, -1.0]","[-1.0, -1.0]","[-456.9526038899921, -456.9526038899921]","[143.14052735166135, 143.14052735166135]","[-77.32792519874533, -77.32792519874533]","[[-456.9526038899921, 143.14052735166135, -77....","[-1.0, -1.0]","[child, child]"
9998,"[False, False]","[36.8669729150927, 9.483383244273043]","[13, -2000001006]","[CC, CC]","[2.2665870947807862, 2.7259379349177366]","[2.8124498056697504, 2.95124214807183]","[-1.0, -1.0]","[-1.0, -1.0]","[-136.56695982955364, -136.56695982955364]","[-477.43635205084996, -477.43635205084996]","[1.6275339020875776, 1.6275339020875776]","[[-136.56695982955364, -477.43635205084996, 1....","[-1.0, -1.0]","[child, child]"


In [7]:
prometheus_set['position'] = prometheus_set['position'].apply(lambda x: [arr.tolist() for arr in x])

In [8]:
prometheus_set

,primary,e,pdg_code,interaction,theta,phi,bjorken_x,bjorken_y,pos_x,pos_y,pos_z,position,column_depth,custom_info
0,"[False, False]","[45.50518012924548, 10.86193159202349]","[13, -2000001006]","[CC, CC]","[2.067075661595442, 2.132042992339948]","[2.5511801288013594, 2.552868822758729]","[-1.0, -1.0]","[-1.0, -1.0]","[-282.10062345064307, -282.10062345064307]","[-235.13657087895137, -235.13657087895137]","[131.77952974883124, 131.77952974883124]","[[-282.10062345064307, -235.13657087895137, 13...","[-1.0, -1.0]","[child, child]"
1,"[False, False]","[1.553154902475276, 18.83828493494808]","[13, -2000001006]","[CC, CC]","[1.257652221818758, 0.8621987961995335]","[-0.07175560329268144, -0.4136971854003417]","[-1.0, -1.0]","[-1.0, -1.0]","[-266.6133132787735, -266.6133132787735]","[392.7532243368922, 392.7532243368922]","[428.3103116051809, 428.3103116051809]","[[-266.6133132787735, 392.7532243368922, 428.3...","[-1.0, -1.0]","[child, child]"
2,"[False, False]","[22.8860991194759, 6.540648101012984]","[14, -2000001006]","[NC, NC]","[1.0591866668147263, 0.9959436495723902]","[1.829713461036645, 1.4610488297749082]","[-1.0, -1.0]","[-1.0, -1.0]","[-190.06969725113595, -190.06969725113595]","[197.51769225223495, 197.51769225223495]","[397.9639690379437, 397.9639690379437]","[[-190.06969725113595, 197.51769225223495, 397...","[-1.0, -1.0]","[child, child]"
3,"[False, False]","[0.6646495645253374, 31.71207236002862]","[14, -2000001006]","[NC, NC]","[1.9034802913019888, 2.5164707547434686]","[0.18215455990949517, -0.2523157785774759]","[-1.0, -1.0]","[-1.0, -1.0]","[-410.56053905574043, -410.56053905574043]","[172.53365903631823, 172.53365903631823]","[220.2599909337447, 220.2599909337447]","[[-410.56053905574043, 172.53365903631823, 220...","[-1.0, -1.0]","[child, child]"
4,"[False, False]","[22.03573692056882, 4.220914829181715]","[14, -2000001006]","[NC, NC]","[1.663206122804835, 1.320453441894709]","[-2.9850900846065826, 3.0611679749533054]","[-1.0, -1.0]","[-1.0, -1.0]","[-101.89797162743382, -101.89797162743382]","[147.4691008193184, 147.4691008193184]","[-199.4574005302444, -199.4574005302444]","[[-101.89797162743382, 147.4691008193184, -199...","[-1.0, -1.0]","[child, child]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,"[False, False]","[27.600063236991506, 3.637189376367924]","[13, -2000001006]","[CC, CC]","[0.5756598304985038, 0.7170793984178334]","[-0.12885668520129273, -0.10395841486766698]","[-1.0, -1.0]","[-1.0, -1.0]","[-311.9725145069959, -311.9725145069959]","[95.77376614555551, 95.77376614555551]","[490.96665858950723, 490.96665858950723]","[[-311.9725145069959, 95.77376614555551, 490.9...","[-1.0, -1.0]","[child, child]"
9996,"[False, False]","[28.423240903483006, 11.059652493790791]","[13, -2000001006]","[CC, CC]","[0.9820697935318712, 0.9132725985913972]","[-2.828069968541139, -2.8870558592753808]","[-1.0, -1.0]","[-1.0, -1.0]","[349.29692694155557, 349.29692694155557]","[-61.512554712957815, -61.512554712957815]","[-285.00734403323037, -285.00734403323037]","[[349.29692694155557, -61.512554712957815, -28...","[-1.0, -1.0]","[child, child]"
9997,"[False, False]","[34.563909769942185, 13.695911597641514]","[13, -2000001006]","[CC, CC]","[1.0678033860163625, 1.0858090499877906]","[-2.3897710301020365, -2.383373459788714]","[-1.0, -1.0]","[-1.0, -1.0]","[-456.9526038899921, -456.9526038899921]","[143.14052735166135, 143.14052735166135]","[-77.32792519874533, -77.32792519874533]","[[-456.9526038899921, 143.14052735166135, -77....","[-1.0, -1.0]","[child, child]"
9998,"[False, False]","[36.8669729150927, 9.483383244273043]","[13, -2000001006]","[CC, CC]","[2.2665870947807862, 2.7259379349177366]","[2.8124498056697504, 2.95124214807183]","[-1.0, -1.0]","[-1.0, -1.0]","[-136.56695982955364, -136.56695982955364]","[-477.43635205084996, -477.43635205084996]","[1.6275339020875776, 1.6275339020875776]","[[-136.56695982955364, -477.43635205084996, 1....","[-1.0, -1.0]","[child, child]"
